Переустановка плагинов (для повторных запусков)

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

In [23]:
!nvidia-smi

Tue Jan  4 15:24:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

# Output would be True if Pytorch is using GPU otherwise it would be False.

True
0
1
Tesla K80


Полностью удалим все предыдущие версии CUDA. Нам нужно обновить облачный экземпляр CUDA.

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Установим CUDA версии 9 

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

Проверим, какая версия CUDA у нас установлена

In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


# Установка расширения для jupyter

Установим расширение CUDA для jupyter

In [6]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-89r22jtd
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-89r22jtd
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=04fbe99f9cf83b557cd3f6d871f277a528b5bbc2b115002a5b032960c989fff4
  Stored in directory: /tmp/pip-ephem-wheel-cache-11t6cd2b/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin


Загрузим расширение

In [7]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


Напишем Hello World, чтобы проверить, что плагин работает

In [7]:
%%cu
#include <iostream>
    int
    main()
{
    std::cout << "Hello World\n";
    return 0;
}

Hello World



# Лабораторная работа №2

 Вариант 6*. 
 Генерация случайных чисел


In [22]:
%%cu
#include <unistd.h>
#include <stdio.h>
#include <curand.h>
#include <curand_kernel.h>

#define N 25

#define MAX 100

/* GPU kernel function to init random states */
__global__ void init(unsigned int seed, curandState_t* states) {

  curand_init(seed,
              blockIdx.x,0, &states[blockIdx.x]);
}

/* this GPU kernel takes an array of states*/
__global__ void randoms(curandState_t* states, unsigned int* numbers) {
  numbers[blockIdx.x] = curand(&states[blockIdx.x]) % 100;
}

int main() {
  curandState_t* states;

  cudaMalloc((void**) &states, N * sizeof(curandState_t));

  init<<<N, 1>>>(time(0), states);

  unsigned int cpu_nums[N];
  unsigned int* gpu_nums;
  cudaMalloc((void**) &gpu_nums, N * sizeof(unsigned int));

  randoms<<<N, 1>>>(states, gpu_nums);

  cudaMemcpy(cpu_nums, gpu_nums, N * sizeof(unsigned int), cudaMemcpyDeviceToHost);

  for (int i = 0; i < N; i++) {
    printf("%u\n", cpu_nums[i]);
  }

  cudaFree(states);
  cudaFree(gpu_nums);

  return 0;
}

50
39
1
95
82
27
31
22
39
35
50
26
59
63
68
20
46
27
12
35
45
73
78
52
81

